In [2]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder

data_folder = '/home/thebatou/Documents/Uni_Potsdam/data_analysis_bitbond/data/'
print 'Loading data ...'
data = pd.read_csv(os.path.join(data_folder, 'data_num_bit.csv'), index_col=0)
print 'Done loading data! {} data were loaded.'.format(data.shape[0])

Loading data ...
Done loading data! 2177 data were loaded.


In [4]:
data.project_description.ix[13]

'Hey everyone, this is a reputation project. Give it a try to see how everything works!'

In [7]:
data.loc[835,'region'] = 'NA'
data.loc[1232,'region'] = 'NA'
data.loc[1839,'region'] = 'NA'
data.loc[1993,'region'] = 'NA'
#data['funded'] = data['funded_at'] != '-1'

In [8]:
for status in np.unique(data['status']):
    print '{} has {} representatives'.format(status, data[data['status']==status].shape[0])

canceled has 881 representatives
charged_off has 119 representatives
current has 59 representatives
defaulted has 49 representatives
expired has 535 representatives
fully_paid has 391 representatives
funded has 73 representatives
in_funding has 11 representatives
late_30 has 10 representatives
late_90 has 49 representatives


In [6]:
categorical_feature = ['loan_identifier', 'borrower_identifier', 'term', 'purpose', 'borrower_rating', 'employment', 'region', 'facebook', 'twitter',
                       'paypal', 'ebay', 'linkedin', 'currency', 'base_currency', 'location']#, 'funded', 'status']
transformed_data = pd.DataFrame()
encoders = []
for feature in categorical_feature:
    le = LabelEncoder()
    encoders.append(le)
    le.fit(np.unique(data[feature]).tolist())
#    print list(le.classes_)
    s = pd.Series(le.transform(data[feature].tolist()))
    s.name = feature
    transformed_data = pd.concat([transformed_data, s], axis=1)

/home/thebatou/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [7]:
from datetime import datetime
from pandas import to_datetime
from datetime import timedelta
# Extracting publication date
published_date = data['published_at']
published_date = published_date.str.rsplit(' ', expand=True, n=1)
published_date = published_date.drop(1, axis=1)
# Exctracting funding date
funded_date = data['funded_at']
funded_date = funded_date.str.rsplit(' ', expand=True, n=1)
funded_date = funded_date.drop(1, axis=1)
funded_date[funded_date[0]=='-1'] = pd.Timestamp.max.strftime('%Y-%m-%d %H:%M:%S')

published_date = to_datetime(published_date[0], format='%Y-%m-%d %H:%M:%S')
funded_date = to_datetime(funded_date[0], format='%Y-%m-%d %H:%M:%S')
diff_date_pub_fund = funded_date - published_date
#print diff_date_pub_fund[diff_date_pub_fund>timedelta(days=90000)].shape

diff_date_pub_fund = diff_date_pub_fund.dt.days

data['time_between_published_funded'] = diff_date_pub_fund
transformed_data['time_between_published_funded'] = diff_date_pub_fund

In [8]:
transformed_data

,loan_identifier,borrower_identifier,term,purpose,borrower_rating,employment,region,facebook,twitter,paypal,ebay,linkedin,currency,base_currency,location,time_between_published_funded
0,2078,51,4,2,1,2,14,1,1,0,1,1,14,0,21,0
1,2079,51,4,5,1,2,14,1,1,0,1,1,14,0,21,2
2,2080,52,3,1,0,2,66,0,0,0,0,0,14,0,60,0
3,2081,52,3,1,0,2,66,0,0,0,0,0,14,0,60,0
4,2082,51,4,3,1,2,14,1,1,0,1,1,14,0,21,0
5,2083,51,3,5,1,2,14,1,1,0,1,1,14,0,21,0
6,2084,51,4,2,0,2,14,1,1,0,1,1,14,0,21,90871
7,2085,482,1,2,4,1,36,1,1,1,1,1,14,0,32,90869
8,2086,484,1,2,3,2,23,1,1,1,1,1,14,0,20,90869
9,2087,52,4,0,0,2,66,0,0,0,0,0,14,0,60,90869


In [11]:
data['ratio_requested_funded'] = data['amount_funded']/data['amount_requested']
transformed_data['ratio_requested_funded'] = data['amount_funded']/data['amount_requested']
transformed_data['amount_funded'] = data['amount_requested']
transformed_data['amount_funded'] = data['amount_funded']
transformed_data['address_lat']= data['address_lat']
transformed_data['address_lng']= data['address_lng']
transformed_data['net_income_cents']= data['net_income_cents']
transformed_data['rates_paid']= data['rates_paid']
transformed_data['rates_count']= data['rates_count']
transformed_data['prices']= data['prices']
transformed_data['portion_average_salary'] = data['net_income_cents']/data['prices']
transformed_data['nominal_interest_rate']= data['nominal_interest_rate']
transformed_data['status']= data['status']
transformed_data.shape

(2177, 27)

In [12]:
print data.shape
print data.columns
print ''
print transformed_data.ix[10]

(2177, 32)
Index([u'loan_identifier', u'nominal_interest_rate', u'term', u'purpose',
       u'project_description', u'borrower_identifier', u'borrower_rating',
       u'employment', u'region', u'facebook', u'twitter', u'paypal', u'ebay',
       u'linkedin', u'published_at', u'funded_at', u'status',
       u'amount_requested', u'amount_funded', u'issuer_id', u'fraudulent',
       u'address_lat', u'address_lng', u'net_income_cents', u'currency',
       u'rates_count', u'rates_paid', u'base_currency', u'location', u'prices',
       u'time_between_published_funded', u'ratio_requested_funded'],
      dtype='object')

loan_identifier                      2088
borrower_identifier                   485
term                                    3
purpose                                 6
borrower_rating                         5
employment                              3
region                                 14
facebook                                1
twitter                                 1
pa

In [86]:
transformed_data.to_csv(path_or_buf='/home/thebatou/Documents/Uni_Potsdam/Competitive_Data_Analysis/transformed_data.csv', encoding='utf-8')